# Fit feedforward Neural Network model

In [ ]:
import warnings

import chart_studio
from besos import eppy_funcs as ef, sampling
from besos.evaluator import EvaluatorEP, EvaluatorGeneric
from besos.problem import EPProblem
from chart_studio import plotly as py
from plotly import graph_objs as go
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

from parameter_sets import parameter_set

We begin by:
+ getting a predefined list of 7 parameters from `parameter_sets.py`
+ making these into a `problem` with electricty use as the objective
+ and making an `evaluator` using the default EnergyPlus building.

In [ ]:
parameters = parameter_set(7)
problem = EPProblem(parameters, ["Electricity:Facility"])
building = ef.get_building()
evaluator = EvaluatorEP(problem, building)

Then we get 20 samples across this design space and evaluate them.

In [ ]:
inputs = sampling.dist_sampler(sampling.lhs, problem, 20)
outputs = evaluator.df_apply(inputs)
inputs

## Train-test split
Next we split the data into a training set (80%) and a testing set (20%).

In [ ]:
train_in, test_in, train_out, test_out = train_test_split(
    inputs, outputs, test_size=0.2
)

## Normalization of inputs

To ensure an equal weighting of inputs and outputs in the backpropagation algorithm fitting the neural network, we have to normalize the input values.
For example window-to-wall ratio is in the range of 0 to 1 while the $W/$m^2$ are in a range of 10 to 15.
Different options for normalization exist.
Here we bring all features (input variables) to have zero mean and a standarddeviation of 1.
Note that we fit the normalizer on training data only.

In [ ]:
scaler = StandardScaler()
inputs = scaler.fit_transform(X=train_in)

scaler_out = StandardScaler()
outputs = scaler_out.fit_transform(X=train_out)

## Hyper-parameters

Before we start fitting the NN model we define the set of hyperparameters we want to analyse in our cross-validation to optimize the model.
Here, we select the number of layers of the network as well as the regularization parameter alpha as parameter value.
A larger number of layers and a lower value of the regularizer lead to higher variance of the network.
This may lead to overfitting.
The best selection may be found using an optimizer like Bayesian Optimization.
In this example we use a simple grid search.

In [ ]:
hyperparameters = {
    "hidden_layer_sizes": (
        (len(parameters) * 16,),
        (len(parameters) * 16, len(parameters) * 16),
    ),
    "alpha": [1, 10, 10 ** 3],
}

neural_net = MLPRegressor(max_iter=1000, early_stopping=False)
folds = 3

## Model fitting

Here, we use the NN model from ScikitLearn.
In a [different example](FitNNTF.ipynb) we use TensorFlow (with and without the Keras wrapper).

In [ ]:
clf = GridSearchCV(neural_net, hyperparameters, iid=True, cv=folds)
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=FutureWarning)
    clf.fit(inputs, outputs.ravel())


print(f"Best performing model $R^2$ score on training set: {clf.best_score_}")
print(f"Model $R^2$ parameters: {clf.best_params_}")
print(
    f"Best performing model $R^2$ score on a separate test set: {clf.best_estimator_.score(scaler.transform(test_in), scaler_out.transform(test_out))}"
)

## Surrogate Modelling Evaluator object
We can wrap the fitted model in a BESOS `Evaluator`.
This has identical behaviour to the original EnergyPlus Evaluator object.

In [ ]:
def evaluation_func(ind, scaler=scaler):
    ind = scaler.transform(X=[ind])
    return (scaler_out.inverse_transform(clf.predict(ind))[0],)


NN_SM = EvaluatorGeneric(evaluation_func, problem)

This has identical behaviour to the original EnergyPlus Evaluator object.
In the next cells we generate a single input sample and evaluate it using the surrogate model and EnergyPlus.

In [ ]:
sample = sampling.dist_sampler(sampling.lhs, problem, 1)
values = sample.values[0]
print(values)

In [ ]:
NN_SM(values)[0]

In [ ]:
evaluator(values)[0]

## Running a large surrogate evaluation

In [ ]:
inputs = sampling.dist_sampler(sampling.lhs, problem, 5000)
outputs = NN_SM.df_apply(inputs)
results = inputs.join(outputs)
results.head()

## Visualization

In [ ]:
chart_studio.tools.set_credentials_file(
    username="besos", api_key="Kb2G2bjOh5gmwh1Midwq"
)
df = inputs.round(3)

# generate list if dictionaries
l = list()
for i in df.columns:
    l.extend([dict(label=i, values=df[i])])

l.extend([dict(label=outputs.columns[0], values=outputs.round(-5))])

data = [
    go.Parcoords(
        line=dict(
            color=outputs["Electricity:Facility"],
            colorscale=[[0, "#D7C16B"], [0.5, "#23D8C3"], [1, "#F3F10F"]],
        ),
        dimensions=l,
    )
]

layout = go.Layout(plot_bgcolor="#E5E5E5", paper_bgcolor="#E5E5E5")

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="parcoords-basic")